# TensorFlow Serving

## Docker example

### Installazione

_docker pull tensorflow/serving_

### Preparazione del modello

In [2]:
import tensorflow.keras as keras

In [3]:
import numpy as np

In [4]:
from random import randint

In [5]:
X = [randint(0,100) for i in range(1000)]

In [6]:
y = [x*2+x/2 for x in X]

In [7]:
model = keras.models.Sequential()

model.add(keras.layers.Dense(100, input_shape=(1,)))
model.add(keras.layers.Dense(1))

model.compile(loss='mae', optimizer='adam', metrics='mae')

In [8]:
model.fit(X,y, epochs=100, verbose=0)

In [9]:
model.predict([10])

array([[25.049393]], dtype=float32)

### Salvataggio del modello

In [10]:
# Nome del modello
name = 'example'

# Versione del modello (TensorFlow Serving gestisce automaticamente l'ultima versione)
version = '003'

# Salvataggio
model.save(filepath=f'models/{name}/{version}', save_format='pb')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
INFO:tensorflow:Assets written to: models/example/003/assets


### Avvio del modello attraverso un'istanza docker

**path dei modelli**

_MODEL_DIR="$(pwd)/models"_

All'interno di questa cartella ci sarà dunque ogni singolo modello in una cartella (example in questo caso), all'interno di ogni cartella ci sarà dunque una sottocartella numerica che conterrà la versione del modello.

**avvio dell'istanza docker per il modello _example_**

docker run -t --rm -p 8501:8501 -v "$MODEL_DIR/example:/models/example" -e MODEL_NAME=example tensorflow/serving &

**esempi di interrogazione attraverso curl**

curl -d "{\"instances\": [[10]]}" -X POST http://localhost:8501/v1/models/example:predict

{
    "predictions": [[24.9380608]
    ]
}

curl -d "{\"instances\": [[10], [10], [20]]}" -X POST http://localhost:8501/v1/models/example:predict

### Interrogazione API attraverso la libreria "requests"

In [11]:
import requests

In [12]:
data = dict()

In [13]:
# data = {"instances": [[10]]}
# data = {"instances": [[10], [20]]}

In [14]:
data['instances'] = [[i] for i in range(1, 3)]
data

{'instances': [[1], [2]]}

In [15]:
url = "http://localhost:8501/v1/models/example:predict"

In [16]:
response = requests.post(url, json=data)

In [17]:
response.status_code

200

In [18]:
response.json()

{'predictions': [[2.50524259], [5.01014853]]}